# Init stuffs

### Imports

In [106]:
import re
import exrex # only used in the bad solution

from functools import lru_cache

### Input data

In [107]:
filename = "./input_1.txt"

with open(file=filename) as f:
    input_data_pt1 = f.read().splitlines()

# Pt. 1

In [108]:
def find_arrangements(configuration: str, hints: list[int]) -> str:
    arrangements = 0

    if len(configuration) == sum(hints)+len(hints)-1:
        return 1
    
    re_pattern = f"-*#{{{hints[0]}}}"
    for number in hints[1:]:
        re_pattern += f"-+#{{{number}}}"
    re_pattern += "-*"
    
    all_possible_strings = [string for string in exrex.generate(s=re_pattern, limit=len(configuration)-(sum(hints)+len(hints)-1)+1) if len(string) == len(configuration)]

    for string in all_possible_strings:
        if re.match(pattern=configuration.replace(".","-").replace("?","."), string=string):
            arrangements += 1

    return arrangements

In [110]:
count = []

for line in input_data_pt1:
    configuration, hints = line.split()
    hint_numbers = [int(x) for x in hints.split(",")]
    count.append(find_arrangements(configuration = configuration, hints = hint_numbers))

sum(count)
    

7407

# Pt. 1 (ver. 2)

In [120]:
@lru_cache(maxsize=500000)
def place_broken_springs(input_configuration: str, groups: tuple[int]) -> int:
    found_matches = 0
    if len(groups) == 1:
        for i in range(len(input_configuration) - groups[0] + 1):
            input_subset = input_configuration[i : i + groups[0]]
            rest = input_configuration[i+groups[0]:]
            if re.match(pattern=rf"[#?]{{{groups[0]}}}", string=input_subset) and (
                input_configuration[i - 1] != "#" if i > 0 else True
            ) and not re.findall(pattern = "#", string = rest) and not re.findall(pattern="#", string = input_configuration[:i]):
                found_matches += 1
        return found_matches

    for i in range(len(input_configuration) - sum(groups) - len(groups) + 2):
        input_subset_plus_one = input_configuration[i : i + groups[0] + 1]
        if re.match(
            pattern=rf"[#?]{{{groups[0]}}}[\.?]", string=input_subset_plus_one
        ) and not re.findall(pattern="#", string = input_configuration[:i]):
            found_matches += place_broken_springs(
                input_configuration=input_configuration[i + groups[0] + 1 :],
                groups=groups[1:],
            )
    return found_matches

In [113]:
solutions = []
for line in input_data_pt1:
    line_configuration, groups = line.split()
    groups = tuple([int(entry) for entry in groups.split(",")])
    solutions.append(place_broken_springs(input_configuration=line_configuration, groups=groups))

sum(solutions)

7407

# Pt. 2

In [ ]:
# NOTE: This does not work (fast enough)
count = 0
for index, line in enumerate(input_data_pt1):
    configuration, hints = line.split()
    configuration = configuration+"?"+configuration+"?"+configuration+"?"+configuration+"?"+configuration
    hints = 5*hints
    count += find_arrangements(configuration = configuration, hints = hint_numbers)
    print(index)

print(count)    

In [121]:
count = []
for index, line in enumerate(input_data_pt1):
    line_configuration, groups = line.split()
    line_configuration = (
        line_configuration
        + "?"
        + line_configuration
        + "?"
        + line_configuration
        + "?"
        + line_configuration
        + "?"
        + line_configuration
    )
    groups = tuple(5*[int(entry) for entry in groups.split(",")])
    count.append(place_broken_springs(input_configuration=line_configuration, groups=groups))

sum(count)

30568243604962

In [122]:
place_broken_springs.cache_info()

CacheInfo(hits=248170, misses=91253, maxsize=500000, currsize=91253)